# Finviz Analytics


### What is Finviz?
FinViz aims to make market information accessible and provides a lot of data in visual snapshots, allowing traders and investors to quickly find the stock, future or forex pair they are looking for. The site provides advanced screeners, market maps, analysis, comparative tools and charts.

### Why?
Leverage the unofficial python API for FinViz to create custom stock screeners to identify value with volatile market conditions.

In [3]:
#!pip install nest_asyncio
#!pip install pycodestyle pycodestyle_magic
#!pip install flake8
 
import pandas as pd 
import finviz
from finviz.screener import Screener
import nest_asyncio

## Load environment and runtime variables



In [4]:
#variable to contain current data module name
module = 'ldg_finviz'

#filters to reduce finviz api payload
#filters = ['exch_nasd', 'idx_sp500', 'fa_div_none']
filters = [] #extract all

#load configs
import os 
d = os.getcwd()
df = d + '\\config\\'
for i in os.listdir(df):
    k = i[:-4] 
    v = eval(open(df + i).read())
    exec("%s=%s" % (k,v))        

## Using Screener

Before using the Screener class, you have to manually go to the website's screener and enter your desired settings. The URL will automatically change every time you add a new setting. After you're done the URL will look something like this:

https://i.imgur.com/p8BLt06.png

Those parameters are a list of key/value pairs separated with the & symbol. Some keys have a clear intent - f=cap_largeover,exch_nasd,fa_fpe_o10 are filters, o=-ticker is order and t=ZM are tickers - yet, some are ambiguous like v=111, which stands for the type of table.

To make matters easier inside the code you won't refer to tables by their number tag, but instead you will use their full name (ex. table=Performance).


In [5]:
def get_data_finviz(filters):
    import nest_asyncio
    nest_asyncio.apply()
    
    env_var, err_var, data_var, transform = unpack_variables(var)
    ldg_path = env_var.get('ldg_path') 
    
    for i in map_api_fv_table_key:
        print(i.lower())
        stock_list = Screener(filters=filters, table=i)
        stock_list.to_csv(ldg_path + 'stock_screener_' + i.lower() + '.csv')

def unpack_variables(var):
    env_var = var['env_var']
    data_var = var['data_var']
    err_var = var['err_var']
    transform = var['transform']
    return env_var, err_var, data_var,transform
    
def log_diagnostics(rpc, e, var):
    print('error',rpc,e,env_var)


def get_transform(target):
    #returns a list of dicts with transform logic in format {field:value, fn:value} 
    lst = [i.get(target) for i in transform['root'] if i.get(target) != None]
    if lst != []: lst = lst[0]
    return lst

#apply transformations to target dataframe
def apply_transform(df, transform):
    for t in transform: 
        fn = t.get('fn')
        if fn[0] == '$': fn = map_generic_fn.get(fn)
        field = t.get('field')
        if field[0] == '[':field = eval(field) 
        print('apply transform: {field, function}',field,fn)
        df[field] = eval(fn)
    return 

#load data from landing, normalize using transforms, output to staging
def normalize_data():
    for i in map_landing_code.get(module):
        #load file for meta contract and data type conversion
        file = ldg_path + i.get('file')
        code = i.get('code')
        df = pd.read_csv(file)

        trns = get_transform(code)
        if trns != []:
            apply_transform(df,trns)
        df.to_csv(stg_path + i.get('file'))
#TODO Normalize column names, tolwower() with underscores...             

def convert_unit(u):
    if len(u) != 1: 
        ua = u
        u = str(u[-1])
        val = str(ua[0:-1]).replace('.','')
    else:  val = ''
    u=u.lower()

    if u == 'm':
        val += '0000'
    elif u == 'b':
        val += '0000000'
    return val 

# Stage Data from FinViz

In [6]:
import numpy as np

#dict containing all relevant runtime variables
var = {'err_var':err_var, 'env_var':env_var,'data_var':data_var,'transform':transform}
env_var, err_var, data_var, transform = unpack_variables(var)
ENF = err_var.get('no_param')
ldg_path = env_var.get('ldg_path') 
stg_path = env_var.get('stg_path')

#get_data_finviz(filters)
#normalize_data()


In [7]:
def enrich_data_finviz(var):
    import numpy as np
    env_var, err_var, data_var, transform = unpack_variables(var)
    
    ENF = err_var.get('no_param')
    stg_path = env_var.get('stg_path')
    
    fown =stg_path + data_var[module].get('stock_ownership',ENF)
    fovr =stg_path + data_var[module].get('stock_overview',ENF)
    key = data_var[module].get('stock_key',ENF)
    
    if (fown==ENF) or (fovr== ENF) or (key == ENF):
        e = 'missing file or key name'
        log_diagnostics('enrich_data_finviz',e,env_var)
        return
    
    #generate additional attibutes
    df_own = pd.read_csv(fown).reset_index()
    df_own.set_index(key,inplace=True)
    df_view = pd.read_csv(fovr).reset_index()
    df_view.set_index(key,inplace=True)
    df = pd.merge(df_own,df_view,how='inner',left_index = True, right_index=True)
    df.reset_index(inplace=True)

                
    transform = get_transform('stg_finviz_summary') 
    apply_transform(df,transform)
    df = df[['Ticker','eps','earnings','P/E','e/p','outstanding_shares']]

    df.to_csv(stg_path + 'stock_screener_summary.csv')


enrich_data_finviz(var)

apply transform: {field, function} Outstanding_unit df.Outstanding.str[-1:].map(convert_unit)
apply transform: {field, function} outstanding_shares (df['Outstanding'].str[:-1].str.replace('.','') + df['Outstanding_unit'])
apply transform: {field, function} outstanding_shares pd.to_numeric(df['outstanding_shares'],errors='coerce').fillna(0)
apply transform: {field, function} P/E (pd.to_numeric(df['P/E'],errors='coerce').fillna(0)).round(decimals=2)
apply transform: {field, function} eps (df['Price_x']/(df['P/E'])).round(decimals=2).replace(np.inf,0)
apply transform: {field, function} earnings ((df['outstanding_shares'].astype(float)) * (df['eps'])).round(decimals=0)
apply transform: {field, function} e/p (df['eps']/df['Price_x']).round(decimals=2)


In [12]:
def generate_docs():
    from pandas_profiling import ProfileReport
    env_var, err_var, data_var, transform = unpack_variables(var)
    stg_path = env_var.get('stg_path')
    ldg_path = env_var.get('ldg_path')
    

    for i in map_landing_code[module]:
        file = i.get('file')
        df = pd.read_csv(ldg_path + file)
        profile = ProfileReport(df, title= 'Profile: ' + file + ' (Landing)')
        profile.to_file(ldg_path + 'profile_' + file[0:-4] + '.html')
        
generate_docs()

In [1]:
#!pip install --user --upgrade joblib
#!pip install pandas-profiling --user
#!pip install <https://github.com/pandas-profiling/pandas-profiling/archive/master.zip> --user
import pandas_profiling as pp
from pandas_profiling import ProfileReport


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
